### **Order allocation algorithm**

Assignment prepared by Chris Reeders
Date last edited: 28/05/2024

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# **Section 1:** Workbook configuration and data imports

In [176]:
pip install haversine

In [249]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [250]:
import pandas as pd

# Read JSON file
allocation_logs_df = pd.read_json('/content/drive/MyDrive/Bash/allocation_algorithm_logs_20240105.json')
dc_branch_numbers_df = pd.read_csv('/content/drive/MyDrive/Bash/dc_branch_numbers.csv', dtype=str)
courier_post_code_mapping_df = pd.read_csv('/content/drive/MyDrive/Bash/courier_post_code_mapping.csv', dtype=str)
courier_costs_df = pd.read_csv('/content/drive/MyDrive/Bash/courier_costs.csv', dtype=str)
branch_fcm_facilities_df = pd.read_csv('/content/drive/MyDrive/Bash/bash_fcm_facilities.csv', dtype=str)

In [251]:
# Display the first few rows of the dataframe
print(allocation_logs_df)

                                                               order_1  \
request              {'destination': {'branch_number': '', 'deliver...   
available_inventory  [{'BranchNumber': '060293', 'CreatedAt': '2023...   
plan                 {'allocation_id': '018cd7ee-fab0-77d9-b417-61f...   

                                                               order_2  \
request              {'destination': {'branch_number': '', 'deliver...   
available_inventory  [{'BranchNumber': '330302', 'CreatedAt': '2023...   
plan                 {'allocation_id': '018cd951-c168-7149-8434-1c6...   

                                                               order_3  \
request              {'destination': {'branch_number': '066019', 'd...   
available_inventory  [{'BranchNumber': '000025', 'CreatedAt': '2023...   
plan                 {'allocation_id': '018cdb42-2356-7261-8393-9f1...   

                                                               order_4  \
request              {'destination'

In [252]:
print(dc_branch_numbers_df)

   branch_number dc_type
0         501870    dc_b
1         060480    dc_b
2         024389    dc_b
3         007305    dc_b
4         060274    dc_b
5         060118    dc_b
6         000818    dc_b
7         004112    dc_b
8         029426    dc_b
9         028078    dc_b
10        330302    dc_b
11        020092    dc_b
12        028184    dc_b
13        023554    dc_b
14        027873    dc_b
15        022886    dc_b
16        018511    dc_b
17        002270    dc_b
18        004950    dc_b
19        038924    dc_b
20        014281    dc_b
21        007081    dc_b
22        456435    dc_b
23        459099    dc_b
24        015868    dc_b
25        013200    dc_b
26        066445    dc_b
27        000070    dc_a
28        000088    dc_a
29        090131    dc_a
30        004373    dc_a
31        016548    dc_a


In [253]:
print(courier_post_code_mapping_df)

      PostalCode HubID       LMR  CourierId
0           0001   PTA     Local  courier_a
1           0002   PTA     Local  courier_a
2           0003   PTA     Local  courier_a
3           0004   PTA     Local  courier_a
4           0005   PTA     Local  courier_a
...          ...   ...       ...        ...
12139       7606   OTD  Regional  courier_b
12140       8259   OTD  Regional  courier_b
12141       8379   OTD  Regional  courier_b
12142       8660   OTD  Regional  courier_b
12143       9960   OTD  Regional  courier_b

[12144 rows x 4 columns]


In [254]:
print(courier_costs_df)

   delivery_type courier_id       lmr   cost
0     DoorToDoor  courier_a     Local     46
1     DoorToDoor  courier_a      Main     69
2     DoorToDoor  courier_a  Regional     87
3     DoorToDoor  courier_b     Local     55
4     DoorToDoor  courier_b      Main     72
5     DoorToDoor  courier_b  Regional     95
6     DoorToDoor  courier_c     Local     20
7     DoorToDoor  courier_c      Main  10000
8     DoorToDoor  courier_c  Regional  10000
9    StorePickup  courier_a     Local     22
10   StorePickup  courier_a      Main     38
11   StorePickup  courier_a  Regional     62
12   StorePickup  courier_b     Local     62
13   StorePickup  courier_b      Main     62
14   StorePickup  courier_b  Regional     62
15   StorePickup  courier_c     Local  10000
16   StorePickup  courier_c      Main  10000
17   StorePickup  courier_c  Regional  10000


In [255]:
print(branch_fcm_facilities_df)

                               facility_id postal_code
0     018a932e-eb07-7b94-9a2e-57c49789d242        0699
1     018a932e-ed02-7d06-8729-9dcf46b16ec6        0699
2     018a932e-ed91-7d99-a93f-a848baea846c        0699
3     018a932e-ee1d-7e27-b06c-275953125398        0699
4     018a932e-ee68-7e6d-9c5e-b7b4ceede07b        0699
...                                    ...         ...
5952  018b4259-3122-778a-ab7d-0651920de669        1684
5953  018a9332-6aeb-7589-9950-0b8a306e731f        6017
5954  018a932e-eefb-7f05-a459-21e0e8bac70b        7945
5955  018a932e-f124-70fc-b2d1-1399450e1608        8001
5956  018a932e-f89e-77a9-bd67-0195484c32b6        7460

[5957 rows x 2 columns]


In [256]:
import pandas as pd
import json

# Function to normalize the order data
def normalize_order_data(order_key, order_data):
    request = order_data['request']
    destination = request['destination']
    items = request['items']
    plan = order_data['plan']
    reservations = plan['reservations']

    normalized_orders = []

    for item in items:
        order_entry = {
            'order_key': order_key,
            'delivery_branch_number': destination['branch_number'],
            'delivery_type': destination['delivery_type'],
            'delivery_latitude': destination['latitude'],
            'delivery_longitude': destination['longitude'],
            'delivery_postal_code': destination['postal_code'],
            'delivery_province': destination['province'],
            'dry_run': request['dry_run'],
            'external_order_reference': request['external_order_reference'],
            'alternative_sku': item['alternative_sku'],
            'alternative_trading_company': item['alternative_trading_company'],
            'excluded_branches': item['excluded_branches'],
            'order_item_id': item['order_item_id'],
            'previous_reservation_ids': item['previous_reservation_ids'],
            'ordered_quantity': item['quantity'],
            'sku': item['sku'],
            'trading_company_number': item['trading_company_number'],
            'order_created_at': request['order_created_at'],
            'order_id': request['order_id'],
        }
        normalized_orders.append(order_entry)

    normalized_inventory = []

    for inventory in order_data['available_inventory']:
        inventory_entry = {
            'order_key': order_key,
            'inventory_branch_number': inventory['BranchNumber'],
            'inventory_created_at': inventory['CreatedAt'],
            'inventory_deleted_at': inventory['DeletedAt'],
            'FacilityActive': inventory['FacilityActive'],
            'FacilityAtCapacity': inventory['FacilityAtCapacity'],
            'FacilityCourierCActive': inventory['FacilityCourierCActive'],
            'FacilityBuffer': inventory['FacilityBuffer'],
            'FacilityCoordinates': inventory['FacilityCoordinates'],
            'FacilityID': inventory['FacilityID'],
            'FacilityProvince': inventory['FacilityProvince'],
            'FacilityRegion': inventory['FacilityRegion'],
            'FacilityReservedQty': inventory['FacilityReservedQty'],
            'FacilitySKUActive': inventory['FacilitySKUActive'],
            'FacilitySubType': inventory['FacilitySubType'],
            'FacilityType': inventory['FacilityType'],
            'ID': inventory['Id'],
            'NormalisedSku': inventory['NormalisedSku'],
            'on_hand_quantity': inventory['OnHandQuantity'],
            'qty_available': inventory['QtyAvailable'],
            'sku': inventory['Sku'],
            'stock_source_updated_at': inventory['StockSourceUpdatedAt'],
            'TradingCompanyNumber': inventory['TradingCompanyNumber'],
            'inventory_updated_at': inventory['UpdatedAt'],
        }
        normalized_inventory.append(inventory_entry)

    normalized_plans = []

    for reservation in reservations:
        plan_entry = {
            'order_key': order_key,
            'allocation_id': plan['allocation_id'],
            'cancellations': plan['cancellations'],
            'created_at': plan['created_at'],
            'dry_run': plan['dry_run'],
            'external_order_reference': plan['external_order_reference'],
            'order_id': plan['order_id'],
            'alternative_sku': reservation['alternative_sku'],
            'alternative_trading_company': reservation['alternative_trading_company'],
            'branch_number': reservation['branch_number'],
            'courier': reservation['courier'],
            'created_at': reservation['created_at'],
            'excluded_branches': reservation['excluded_branches'],
            'fulfilment_centre_code': reservation['omni_fulfilment_data']['fulfilment_centre_code'],
            'fulfilment_centre_description': reservation['omni_fulfilment_data']['fulfilment_centre_description'],
            'fulfilment_centre_id': reservation['omni_fulfilment_data']['fulfilment_centre_id'],
            'order_item_id': reservation['order_item_id'],
            'previous_reservation_ids': reservation['previous_reservation_ids'],
            'quantity': reservation['quantity'],
            'reservation_id': reservation['reservation_id'],
            'sku': reservation['sku'],
            'trading_company_number': reservation['trading_company_number']
        }
        normalized_plans.append(plan_entry)

    return normalized_orders, normalized_inventory, normalized_plans

# Initialize lists to hold all normalized data
all_orders = []
all_inventory = []
all_plans = []

# Iterate through each order in the JSON data
for order_key, order_data in allocation_logs_df.items():
    normalized_orders, normalized_inventory, normalized_plans = normalize_order_data(order_key, order_data)
    all_orders.extend(normalized_orders)
    all_inventory.extend(normalized_inventory)
    all_plans.extend(normalized_plans)

# Convert the lists to DataFrames
orders_df = pd.DataFrame(all_orders)
inventory_df = pd.DataFrame(all_inventory)
plans_df = pd.DataFrame(all_plans)

# Join normalized_orders and normalized_inventory on order_key, sku, and, and trading company number
order_inventory_df = pd.merge(inventory_df, orders_df, left_on=['order_key', 'sku', 'TradingCompanyNumber'], right_on=['order_key', 'sku', 'trading_company_number'], how='inner')



In [257]:
# Save to CSV files (optional)
orders_df.to_csv('/content/drive/MyDrive/Colab/Output/orders.csv', index=False)
inventory_df.to_csv('/content/drive/MyDrive/Colab/Output/inventory.csv', index=False)
plans_df.to_csv('/content/drive/MyDrive/Colab/Output/plans.csv', index=False)
order_inventory_df.to_csv('/content/drive/MyDrive/Colab/Output/orders_inventory.csv', index=False)

In [258]:
# Display the first few rows of the joined DataFrame
print(order_inventory_df.columns.tolist())
print(order_inventory_df.loc[order_inventory_df['order_key'] == 'order_10'])
#print(order_inventory_df)

['order_key', 'inventory_branch_number', 'inventory_created_at', 'inventory_deleted_at', 'FacilityActive', 'FacilityAtCapacity', 'FacilityCourierCActive', 'FacilityBuffer', 'FacilityCoordinates', 'FacilityID', 'FacilityProvince', 'FacilityRegion', 'FacilityReservedQty', 'FacilitySKUActive', 'FacilitySubType', 'FacilityType', 'ID', 'NormalisedSku', 'on_hand_quantity', 'qty_available', 'sku', 'stock_source_updated_at', 'TradingCompanyNumber', 'inventory_updated_at', 'delivery_branch_number', 'delivery_type', 'delivery_latitude', 'delivery_longitude', 'delivery_postal_code', 'delivery_province', 'dry_run', 'external_order_reference', 'alternative_sku', 'alternative_trading_company', 'excluded_branches', 'order_item_id', 'previous_reservation_ids', 'ordered_quantity', 'trading_company_number', 'order_created_at', 'order_id']
   order_key inventory_branch_number         inventory_created_at  \
64  order_10                  000084  2023-10-08T16:55:10.072146Z   
65  order_10                 

In [259]:
# Display the DataFrame
print("\nOrders")
print(orders_df.loc[orders_df['order_key'] == 'order_10'])


Orders
   order_key delivery_branch_number delivery_type  delivery_latitude  \
11  order_10                           DoorToDoor                NaN   
12  order_10                           DoorToDoor                NaN   

    delivery_longitude delivery_postal_code delivery_province  dry_run  \
11                 NaN                 3242     KwaZulu-Natal    False   
12                 NaN                 3242     KwaZulu-Natal    False   

   external_order_reference alternative_sku alternative_trading_company  \
11              B4409121-01            None                          99   
12              B4409121-01            None                        None   

   excluded_branches                         order_item_id  \
11              None  dbb15e32-3520-556b-a0bf-ced6f93334cd   
12              None  33a1a1fc-f783-55fc-b6be-a181793952cf   

   previous_reservation_ids  ordered_quantity       sku  \
11                     None                 1  56258808   
12                   

In [260]:
# Display the DataFrame
print("\nInventory")
print(inventory_df.loc[inventory_df['order_key'] == 'order_10'])


Inventory
   order_key inventory_branch_number         inventory_created_at  \
64  order_10                  000084  2023-10-08T16:55:10.072146Z   
65  order_10                  000522  2023-10-08T17:04:29.005508Z   
66  order_10                  005685  2023-10-07T08:48:41.397401Z   
67  order_10                  005886  2023-10-08T16:55:10.101096Z   
68  order_10                  000922  2023-10-10T08:19:21.091664Z   

   inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
64                 None            True               False   
65                 None            True               False   
66                 None            True               False   
67                 None            True               False   
68                 None            True               False   

    FacilityCourierCActive  FacilityBuffer  \
64                   False               1   
65                   False               1   
66                   False               1   
67         

In [261]:
# Display the DataFrame
print("\nPlans")
print(plans_df.columns.tolist())
print(plans_df.head())


Plans
['order_key', 'allocation_id', 'cancellations', 'created_at', 'dry_run', 'external_order_reference', 'order_id', 'alternative_sku', 'alternative_trading_company', 'branch_number', 'courier', 'excluded_branches', 'fulfilment_centre_code', 'fulfilment_centre_description', 'fulfilment_centre_id', 'order_item_id', 'previous_reservation_ids', 'quantity', 'reservation_id', 'sku', 'trading_company_number']
  order_key                         allocation_id cancellations  \
0   order_1  018cd7ee-fab0-77d9-b417-61fd0f96e865          None   
1   order_2  018cd951-c168-7149-8434-1c66c1a9e5a0          None   
2   order_3  018cdb42-2356-7261-8393-9f18149c39d4          None   
3   order_4  018cda6a-5617-76ff-965b-2de978427181          None   
4   order_5  018cd939-895f-7fea-a0e3-0c79f800f6ab          None   

                       created_at  dry_run external_order_reference  \
0  2024-01-05T04:43:32.443897453Z    False              B4403050-01   
1  2024-01-05T11:11:03.068050754Z    False   

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# **Section 2:** Data preparation

In [301]:
# Copy inventory_df
order_inventory = order_inventory_df

In [302]:
# Find postal code for each collection facility and add to order_inventory
order_inventory = pd.merge(order_inventory, branch_fcm_facilities_df, how='left', left_on='FacilityID', right_on='facility_id')
order_inventory = order_inventory.rename(columns={'postal_code': 'collection_postal_code'})

# Display updated order_inventory dataframe
print(order_inventory)

         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105768  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105769  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105770  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105771  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105772  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [303]:
# Find LMR and HubID for each courier based on the delivery postal code and postal code of the collection facility

# Select post code mapping for courier a
post_code_mapping_courier_a = courier_post_code_mapping_df.loc[courier_post_code_mapping_df['CourierId'] == 'courier_a']
# Get mapping for courier a
order_inventory = pd.merge(order_inventory, post_code_mapping_courier_a, how='left', left_on='delivery_postal_code', right_on='PostalCode')
# Rename columns to specify that the LMR and Hub ID are associated with courier a
order_inventory = order_inventory.rename(columns={'LMR': 'LMR_delivery_courier_a', 'HubID': 'HubID_delivery_courier_a' })
# Drop columns that are not needed
order_inventory = order_inventory.drop(columns=['PostalCode', 'CourierId'])
# Get mapping for courier a
order_inventory = pd.merge(order_inventory, post_code_mapping_courier_a, how='left', left_on='collection_postal_code', right_on='PostalCode')
# Rename columns to specify that the LMR and Hub ID are associated with courier a
order_inventory = order_inventory.rename(columns={'LMR': 'LMR_collection_courier_a', 'HubID': 'HubID_collection_courier_a' })
# Drop columns that are not needed
order_inventory = order_inventory.drop(columns=['PostalCode', 'CourierId'])

# Select post code mapping for courier b
post_code_mapping_courier_b = courier_post_code_mapping_df.loc[courier_post_code_mapping_df['CourierId'] == 'courier_b']
# Get mapping for courier b
order_inventory = pd.merge(order_inventory, post_code_mapping_courier_b, how='left', left_on='delivery_postal_code', right_on='PostalCode')
# Rename columns to specify that the LMR and Hub ID are associated with courier b
order_inventory = order_inventory.rename(columns={'LMR': 'LMR_delivery_courier_b', 'HubID': 'HubID_delivery_courier_b' })
# Drop columns that are not needed
order_inventory = order_inventory.drop(columns=['PostalCode', 'CourierId'])
# Get mapping for courier b
order_inventory = pd.merge(order_inventory, post_code_mapping_courier_b, how='left', left_on='collection_postal_code', right_on='PostalCode')
# Rename columns to specify that the LMR and Hub ID are associated with courier b
order_inventory = order_inventory.rename(columns={'LMR': 'LMR_collection_courier_b', 'HubID': 'HubID_collection_courier_b' })
# Drop columns that are not needed
order_inventory = order_inventory.drop(columns=['PostalCode', 'CourierId'])

# Select post code mapping for courier c
post_code_mapping_courier_c = courier_post_code_mapping_df.loc[courier_post_code_mapping_df['CourierId'] == 'courier_c']
# Get mapping for courier c
order_inventory = pd.merge(order_inventory, post_code_mapping_courier_c, how='left', left_on='delivery_postal_code', right_on='PostalCode')
# Rename columns to specify that the LMR and Hub ID are associated with courier c
order_inventory = order_inventory.rename(columns={'LMR': 'LMR_delivery_courier_c', 'HubID': 'HubID_delivery_courier_c' })
# Drop columns that are not needed
order_inventory = order_inventory.drop(columns=['PostalCode', 'CourierId'])
# Get mapping for courier c
order_inventory = pd.merge(order_inventory, post_code_mapping_courier_c, how='left', left_on='collection_postal_code', right_on='PostalCode')
# Rename columns to specify that the LMR and Hub ID are associated with courier c
order_inventory = order_inventory.rename(columns={'LMR': 'LMR_collection_courier_c', 'HubID': 'HubID_collection_courier_c' })
# Drop columns that are not needed
order_inventory = order_inventory.drop(columns=['PostalCode', 'CourierId'])

# Display updated order_inventory dataframe
print(order_inventory)

         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [304]:
# Find DC Type for each inventory branch

order_inventory = pd.merge(order_inventory, dc_branch_numbers_df, how='left', left_on='inventory_branch_number', right_on='branch_number')
order_inventory['dc_type'] = order_inventory['dc_type'].fillna('STORE')
order_inventory = order_inventory.drop(columns=['branch_number'])

# Display updated order_inventory dataframe
print(order_inventory)

         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [305]:
# Check if the facility has enough avialable inventory to fulful the order. Flag facilites that don't have suffiecient inventory to fulfull an order.
# This is used when allocating orders to avoid picking the same item from multiple facilities.
def inventory_quantity_check(row):
    if row['qty_available'] >= row['ordered_quantity']:
        return True
    else:
        return False

order_inventory['sufficient_inventory'] = order_inventory.apply(inventory_quantity_check, axis=1)

# Display updated order_inventory dataframe
print(order_inventory)
# print(order_inventory.loc[order_inventory['sufficient_inventory'] == True])

         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [306]:
# Implement logic for overall route classification

# Add overall route classification for courier a
def route_classification_courier_a(row):
    if row['LMR_delivery_courier_a'] == 'Regional' or row['LMR_collection_courier_a'] == 'Regional':
        return 'Regional'
    elif row['HubID_collection_courier_a'] != row['HubID_delivery_courier_a'] and row['LMR_delivery_courier_a'] != 'Regional':
        return 'Main'
    elif row['HubID_collection_courier_a'] != row['HubID_delivery_courier_a'] and row['LMR_collection_courier_a'] != 'Regional':
        return 'Main'
    elif row['HubID_collection_courier_a'] == row['HubID_delivery_courier_a'] and row['LMR_delivery_courier_a'] != 'Regional':
        return 'Local'
    elif row['HubID_collection_courier_a'] == row['HubID_delivery_courier_a'] and row['LMR_collection_courier_a'] != 'Regional':
        return 'Local'
    else:
        return 'Other'

order_inventory['courier_a_LMR'] = order_inventory.apply(route_classification_courier_a, axis=1)

# Add overall route classification for courier b
def route_classification_courier_b(row):
    return row['LMR_delivery_courier_b']

order_inventory['courier_b_LMR'] = order_inventory.apply(route_classification_courier_b, axis=1)

# Add overall route classification for courier c
def route_classification_courier_c(row):
    if row['LMR_delivery_courier_c'] == 'Regional' or row['LMR_collection_courier_c'] == 'Regional':
        return 'Regional'
    elif row['HubID_collection_courier_c'] != row['HubID_delivery_courier_c'] and row['LMR_delivery_courier_c'] != 'Regional':
        return 'Main'
    elif row['HubID_collection_courier_c'] != row['HubID_delivery_courier_c'] and row['LMR_collection_courier_c'] != 'Regional':
        return 'Main'
    elif row['HubID_collection_courier_c'] == row['HubID_delivery_courier_c'] and row['LMR_delivery_courier_c'] != 'Regional':
        return 'Local'
    elif row['HubID_collection_courier_c'] == row['HubID_delivery_courier_c'] and row['LMR_collection_courier_c'] != 'Regional':
        return 'Local'
    else:
        return 'Other'

order_inventory['courier_c_LMR'] = order_inventory.apply(route_classification_courier_c, axis=1)

# Display updated order_inventory dataframe
print(order_inventory)

         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [307]:
# Calculate distance between collection facility and delivery address

import numpy as np
import struct
import binascii
import geopandas as gpd
from shapely import wkb
from haversine import haversine, Unit

# Function to convert WKB hex to coordinates
def wkb_hex_to_coordinates(wkb_hex):
    if not wkb_hex:
        return None, None
    wkb_bin = binascii.unhexlify(wkb_hex)
    geom = wkb.loads(wkb_bin)
    return geom.y, geom.x

# Apply the WKB hex conversion function and split the result into two columns
coordinates = order_inventory['FacilityCoordinates'].apply(lambda x: wkb_hex_to_coordinates(x) if pd.notnull(x) else (None, None))
order_inventory['collection_latitude'], order_inventory['collection_longitude'] = zip(*coordinates)
order_inventory['collection_coordinates'] = order_inventory.apply(lambda row: (row['collection_latitude'], row['collection_longitude']), axis=1)
order_inventory['delivery_coordinates']= order_inventory.apply(lambda row: (row['delivery_latitude'], row['delivery_longitude']), axis=1)
#print(order_inventory.columns.tolist())

# Define distance calculation using haversine
def calculate_distance(row):
    return haversine(row['collection_coordinates'], row['delivery_coordinates'], unit=Unit.KILOMETERS)

# Apply the distance calculation
order_inventory['distance_km'] = order_inventory.apply(calculate_distance, axis=1)

# Display updated order_inventory dataframe
print(order_inventory)

         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [308]:
# Implement courier availability logic

# Courier a does not collect from all facilities except dc_b
def availability_courier_a(row):
    if row['dc_type'] == 'dc_b':
        return False
    else:
        return True
order_inventory['availability_courier_a'] = order_inventory.apply(availability_courier_a, axis=1)

# Courier b only fulfils orders from dc_b facilities
def availability_courier_b(row):
    if row['dc_type'] == 'dc_b':
        return True
    else:
        return False
order_inventory['availability_courier_b'] = order_inventory.apply(availability_courier_b, axis=1)

# Courier c only fulfils orders that are within from facilities flagged as active for courier c (FacilitiyCourierCActive == True)
def availability_courier_c(row):
    if row['FacilityCourierCActive'] == True and row['distance_km'] <= 20:
        return True
    else:
        return False
order_inventory['availability_courier_c'] = order_inventory.apply(availability_courier_c, axis=1)

# Display updated order_inventory dataframe
print(order_inventory)

         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [309]:
# Get courier costs (from courier_costs_df)

# Split courier_cost_df by courier ID
costs_courier_a = courier_costs_df.loc[courier_costs_df['courier_id'] == 'courier_a']
costs_courier_a = costs_courier_a.drop(columns=['courier_id'])
costs_courier_b = courier_costs_df.loc[courier_costs_df['courier_id'] == 'courier_b']
costs_courier_b = costs_courier_b.drop(columns=['courier_id'])
costs_courier_c = courier_costs_df.loc[courier_costs_df['courier_id'] == 'courier_c']
costs_courier_c = costs_courier_c.drop(columns=['courier_id'])

# Get costs for courier a
order_inventory = pd.merge(order_inventory, costs_courier_a, how='left', left_on=['delivery_type', 'courier_a_LMR'], right_on=['delivery_type', 'lmr'])
order_inventory = order_inventory.rename(columns={'cost': 'cost_courier_a'})
order_inventory['cost_courier_a'] = order_inventory['cost_courier_a'].astype(float)
order_inventory = order_inventory.drop(columns=['lmr'])

# Get costs for courier b
order_inventory = pd.merge(order_inventory, costs_courier_b, how='left', left_on=['delivery_type', 'courier_b_LMR'], right_on=['delivery_type', 'lmr'])
order_inventory = order_inventory.rename(columns={'cost': 'cost_courier_b'})
order_inventory['cost_courier_b'] = order_inventory['cost_courier_b'].astype(float)
order_inventory = order_inventory.drop(columns=['lmr'])

# Get costs for courier C
order_inventory = pd.merge(order_inventory, costs_courier_c, how='left', left_on=['delivery_type', 'courier_c_LMR'], right_on=['delivery_type', 'lmr'])
order_inventory = order_inventory.rename(columns={'cost': 'cost_courier_c'})
order_inventory['cost_courier_c'] = order_inventory['cost_courier_c'].astype(float)
order_inventory = order_inventory.drop(columns=['lmr'])

# Display updated order_inventory dataframe
print(order_inventory)


         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [310]:
# For order_inventory, for each order_id, if inventory_branch is found in excluded_branches, flag the row as 'True' in a new column called "excluded"
order_inventory["excluded"] = np.where(order_inventory['inventory_branch_number'].isin(order_inventory.groupby('order_id')['excluded_branches'].apply(lambda x: x.str.split(', ').sum())), True, False)

# Filter out excluded branches
order_inventory = order_inventory.loc[order_inventory['excluded'] == False]
print(order_inventory)


         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [312]:
order_inventory_data_df = order_inventory

# Save order_inventory dataframe as .csv
order_inventory_data_df.to_csv('/content/drive/MyDrive/Colab/Output/order_inventory_data.csv', index=False)

order_inventory contains all the data required to optimse allocation costs.

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>

#**Section 3:** Bundling

Items allocated to the same facility are dundled into a single parcel. Reducing the number of facilities used to fulful an order will reduce overall courier costs.

This section allocates items to facilities is such a what that the number of facilities used to fulfil an order is minimised.

In [313]:
# Copy order_inventory_data_df
order_inventory_data = order_inventory_data_df

In [314]:
# Find the branch which can fulfil the most items for each order

# Count the number of order_item_id per inventory_branch for each order_id
item_count_by_branch = order_inventory_data.groupby(['order_key', 'order_id', 'inventory_branch_number']).size().to_frame(name='item_count')
item_count_by_branch = item_count_by_branch.reset_index()
item_count_by_branch = item_count_by_branch.sort_values(by=['order_key', 'item_count'], ascending=[True, False])

#print(item_count_by_branch)
#print(item_count_by_branch.loc[item_count_by_branch['order_key'] == 'order_3779'])

#
branch_max_items = item_count_by_branch.loc[item_count_by_branch.groupby(['order_key', 'order_id'])['item_count'].idxmax()]
branch_max_items = branch_max_items.sort_values(by=['item_count'], ascending=[False])
branch_max_items = branch_max_items.rename(columns={'item_count': 'item_count_max'})
branch_max_items['max_items_flag'] = True
print(branch_max_items)


        order_key                              order_id  \
59166  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
70497   order_866  336ac559-db4c-5bc0-9306-d27e8ab63126   
29763  order_2382  5040060c-b8f7-5e44-bf06-be1ffcac5e09   
4737   order_1215  ac64ea13-6d06-5e59-ada8-0fe1129d714d   
35529  order_2646  265df23c-efe0-572e-a2e5-88d38965f1d1   
...           ...                                   ...   
29693  order_2379  f818776a-6dcd-5410-aea9-8f52285016ca   
29694   order_238  30591c7f-fb95-5480-a881-ae3459dfdd08   
29740  order_2381  9a8a288f-af27-50a0-8fe8-3716d1ee1f86   
29766  order_2383  f88ac911-03c6-53b9-934e-b86429c8e58d   
73281   order_999  7e533fbd-cf39-5530-8502-cedb06042ec4   

      inventory_branch_number  item_count_max  max_items_flag  
59166                  038924              20            True  
70497                  330302              18            True  
29763                  330302              14            True  
4737                   038924      

In [315]:
# Flag all item order IDs on the branch max item list
allocation_new = pd.merge(order_inventory_data, branch_max_items, how='left', left_on=['order_key', 'order_id', 'inventory_branch_number'], right_on=['order_key', 'order_id', 'inventory_branch_number'])
allocation_new = allocation_new.drop(columns=['item_count_max'])
print(allocation_new.loc[allocation_new['order_key'] == 'order_3779'])

         order_key inventory_branch_number         inventory_created_at  \
104601  order_3779                  038924   2023-10-09T13:48:04.88974Z   
104602  order_3779                  038924  2023-10-07T10:45:43.038912Z   
104603  order_3779                  030682   2023-10-07T12:44:14.47417Z   
104604  order_3779                  030658  2023-10-07T10:46:35.825246Z   
104605  order_3779                  030667  2023-10-10T08:47:31.957431Z   
...            ...                     ...                          ...   
104813  order_3779                  030310  2023-11-22T06:52:43.471418Z   
104814  order_3779                  030310  2023-10-24T13:51:19.039283Z   
104815  order_3779                  030460  2023-10-09T20:53:39.522126Z   
104816  order_3779                  035854  2023-10-13T20:13:48.497413Z   
104817  order_3779                  030993  2023-10-08T05:53:31.855995Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
104601                 None     

In [316]:
# Extract a list of all the order items which have been flagged as max items.
# Flag these items against other fulfulment facilities and remove them as potential options.
# This will ensure that all items flagged as max items get picked from the same fulfulment facilities.

allocated_order_items = allocation_new.loc[allocation_new['max_items_flag'] == True]

allocated_order_items = allocated_order_items[['order_key', 'order_id', 'order_item_id', 'sku']].drop_duplicates()

allocated_order_items['allocated_flag'] = True

#print(allocated_order_items)

allocation_new = pd.merge(allocation_new, allocated_order_items, how='left', left_on=['order_key', 'order_id', 'order_item_id', 'sku'], right_on=['order_key', 'order_id', 'order_item_id', 'sku'])

allocation_new_update = allocation_new

allocation_new_update['max_items_flag'] = allocation_new_update['max_items_flag'].fillna(False)
#print(allocation_new_update)

allocation_new_final = allocation_new_update.loc[(allocation_new_update['allocated_flag'] == True) & (allocation_new_update['max_items_flag'] == False)]
print(allocation_new_final)

         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [317]:
allocation_new_df = allocation_new_final

# Save order_inventory dataframe as .csv
allocation_new_df.to_csv('/content/drive/MyDrive/Colab/Output/allocation_new.csv', index=False)

## **Section 4.** Calculate the cost based on the new allocation algo

Simplifying assumptions:
In cases where multiple units of an item are ordered, collect all units from the same facility.


In [319]:
allocation_data = allocation_new_df

In [320]:
# Find courier with lowest, valid cost

# Adjust courier cost to 999999 if courier not available
def courier_a_cost_adjusted(row):
    if row['availability_courier_a'] == False:
        return 999999
    else:
        return row['cost_courier_a']

allocation_data['cost_adj_courier_a'] = allocation_data.apply(courier_a_cost_adjusted, axis=1)

# Adjust courier cost to 999999 if courier not available
def courier_b_cost_adjusted(row):
    if row['availability_courier_b'] == False:
        return 999999
    else:
        return row['cost_courier_b']

allocation_data['cost_adj_courier_b'] = allocation_data.apply(courier_b_cost_adjusted, axis=1)

# Adjust courier cost to 999999 if courier not available
def courier_c_cost_adjusted(row):
    if row['availability_courier_c'] == False:
        return 999999
    else:
        return row['cost_courier_c']

allocation_data['cost_adj_courier_c'] = allocation_data.apply(courier_c_cost_adjusted, axis=1)

allocation_data['min_cost'] = allocation_data[['cost_adj_courier_a', 'cost_adj_courier_b', 'cost_adj_courier_c']].min(axis=1)

allocation_data['min_cost_courier'] = allocation_data[['cost_adj_courier_a', 'cost_adj_courier_b', 'cost_adj_courier_c']].idxmin(axis=1)

#cut min_cost_courier description
def cut_string(string, n):
  return string[n:]

allocation_data['min_cost_courier'] = allocation_data['min_cost_courier'].apply(cut_string, args=(9,))

# Display updated order_inventory dataframe
print(allocation_data)

         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [321]:
# Identify cases where the delivery branch and the inventory branch are the same. Set min_cost_courier to same_store and set min_cost to 0

def same_store(row):
    if row['delivery_branch_number'] == row['inventory_branch_number']:
        return 'same_store'
    else:
        return row['min_cost_courier']

allocation_data['min_cost_courier'] = allocation_data.apply(same_store, axis=1)

def same_store_cost(row):
    if row['delivery_branch_number'] == row['inventory_branch_number']:
        return 0
    else:
        return row['min_cost']

allocation_data['min_cost'] = allocation_data.apply(same_store_cost, axis=1)

# Display updated order_inventory dataframe
print(allocation_data)
#print(order_inventory.loc[order_inventory['min_cost_courier'] == 'same_store'])


         order_key inventory_branch_number         inventory_created_at  \
0          order_1                  060293  2023-10-12T07:55:06.692344Z   
1          order_1                  060805  2023-10-12T07:54:52.848081Z   
2          order_1                  060987  2023-10-12T07:54:51.088767Z   
3          order_1                  060853  2023-10-12T07:54:47.379074Z   
4          order_1                  060932  2023-10-12T07:54:52.410766Z   
...            ...                     ...                          ...   
105779  order_3810                  000678   2023-10-08T10:59:40.91907Z   
105780  order_3810                  000720  2023-10-09T09:22:03.425223Z   
105781  order_3810                  000437  2023-10-08T16:58:15.312794Z   
105782  order_3810                  000057  2023-10-09T00:25:19.521028Z   
105783  order_3810                  000025  2023-10-06T16:07:25.591262Z   

       inventory_deleted_at  FacilityActive  FacilityAtCapacity  \
0                      None     

In [330]:
# For each distinct item in each order in the filtered allocation dataframe, flag the branch which corresponds with the lowest courier cost
min_cost_pivot = pd.pivot_table(allocation_data, values='min_cost', index=['order_key', 'order_id', 'order_item_id', 'sku'], aggfunc='min')
min_cost_reset = min_cost_pivot.reset_index()
# Set min cost collection facility flag to 1 for these rows
min_cost_reset['min_cost_collection_facility'] = 1
min_cost_reset = min_cost_reset.sort_values(by=['sku'])
print(min_cost_reset.loc[min_cost_reset['order_key'] == 'order_3779'])

allocation_data = pd.merge(allocation_data, min_cost_reset, how='left', left_on=['order_key', 'order_id', 'order_item_id', 'sku'], right_on=['order_key', 'order_id', 'order_item_id', 'sku'])


#print(allocation_new_final)
#print(order_inventory.loc[new_order_inventory['order_key'] == 'order_10'])
allocation_data = allocation_data.sort_values(by=['sku'])
print(allocation_data.loc[allocation_data['order_key'] == 'order_3779'])


       order_key                              order_id  \
6929  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6916  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6924  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6922  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6936  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6954  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6915  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6950  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6944  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6932  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6914  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6921  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6917  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6952  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6918  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6933  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6911  order_37

In [331]:
# Filter for lowest courier cost
allocation_min_cost = allocation_data.loc[allocation_data['min_cost_collection_facility'] == 1]
#print(allocation_min_cost.loc[allocation_min_cost['order_key'] == 'order_3779'])

# Get the courier cost for each batch of itmes colleced per branch
allocation_min_cost_pivot = pd.pivot_table(allocation_min_cost, values='min_cost', index=['order_key', 'order_id', 'inventory_branch_number', 'min_cost_courier'], aggfunc='min', fill_value=0)

# Reset the index to have a flat DataFrame
allocation_cost_reset = allocation_min_cost_pivot.reset_index()
#print("\nPlans Cost Pivot Reset")
#print(plans_pivot_cost_reset)
print(allocation_cost_reset.loc[allocation_cost_reset['order_key'] == 'order_3779'])

# Get the list of items (SKUs) assoicated with each branch
allocation_sku_pivot = pd.pivot_table(allocation_min_cost, values='sku', index=['order_key', 'order_id', 'inventory_branch_number', 'min_cost_courier'], aggfunc=lambda x: ', '.join(x), fill_value=0)

# Reset the index to have a flat DataFrame
allocation_sku_reset = allocation_sku_pivot.reset_index()
#print("\nPlans SKUs Pivot Reset")
#print(plans_pivot_sku_reset)
print(allocation_sku_reset.loc[allocation_sku_reset['order_key'] == 'order_3779'])

# Join the two pivot tables to get the costs and the SKUs for each order
allocation_by_branch_new = pd.merge(allocation_cost_reset, allocation_sku_reset, how='right', left_on=['order_key', 'order_id', 'inventory_branch_number', 'min_cost_courier'], right_on=['order_key', 'order_id', 'inventory_branch_number', 'min_cost_courier'])

#print(allocation_by_branch_new)
#print(allocation_by_branch_new.loc[allocation_by_branch_new['order_key'] == 'order_3779'])


       order_key                              order_id  \
6134  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6135  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6136  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6137  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6138  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6139  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6140  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6141  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6142  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6143  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6144  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6145  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6146  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   
6147  order_3779  350da824-8efc-545a-8b38-66198acc6ccd   

     inventory_branch_number min_cost_courier  min_cost  
6134                  030310        courier_a        38  
6135         

In [332]:
# Save to CSV files (optional)
allocation_by_branch_new.to_csv('/content/drive/MyDrive/Colab/Output/allocation_plans_new.csv', index=False)

#**Section 5. Calculate the allocation costs for the current allocation algo**

In [282]:
# Get courier costs for existing allocation plan algo

# Get delivery type for each order
order_id_delivery_type = orders_df[['order_id', 'delivery_type']].drop_duplicates()
plans = pd.merge(plans_df, order_id_delivery_type, how='left', left_on='order_id', right_on='order_id')
#plans = plans.drop(columns=['order_id'])
#print(plans)

# Get postal codes for each collection branch/facility
branch_postal_codes = order_inventory[['inventory_branch_number', 'collection_postal_code']].drop_duplicates()
plans = pd.merge(plans, branch_postal_codes, how='left', left_on=['branch_number'], right_on=['inventory_branch_number'])
plans = plans.drop(columns=['inventory_branch_number'])
#plans = plans.rename(columns={'collection_postal_code': 'postal_code'})
#print(plans)

# Get collection Hub ID and LMR for each  branch/facility
#postal_code_hubs = courier_post_code_mapping_df[['PostalCode', 'HubID']].drop_duplicates()
plans = pd.merge(plans, courier_post_code_mapping_df, how='left', left_on=['collection_postal_code', 'courier'], right_on=['PostalCode', 'CourierId'])
plans = plans.drop(columns=['PostalCode', 'CourierId'])
plans = plans.rename(columns={'HubID': 'HubID_collection', 'LMR':'LMR_collection'})
#print(plans)

# Get postal codes for each delivery branch/facility
order_id_postal_codes = orders_df[['order_id', 'delivery_postal_code']].drop_duplicates()
plans = pd.merge(plans, order_id_postal_codes, how='left', left_on='order_id', right_on='order_id')
plans = plans.drop(columns=['order_id'])
#print(plans)

# Get delivery Hub ID and LMR for each branch/facility
plans = pd.merge(plans, courier_post_code_mapping_df, how='left', left_on=['delivery_postal_code', 'courier'], right_on=['PostalCode', 'CourierId'])
plans = plans.drop(columns=['PostalCode', 'CourierId'])
plans = plans.rename(columns={'HubID': 'HubID_delivery', 'LMR':'LMR_delivery'})
#print(plans)

# Get overall LMR for each allocation
def route_classification(row):
    if row['courier'] == 'courier_a' or row['courier'] == 'courier_c':
        if row['LMR_delivery'] == 'Regional' or row['LMR_collection'] == 'Regional':
            return 'Regional'
        elif row['HubID_collection'] != row['HubID_delivery'] and row['LMR_delivery'] != 'Regional':
            return 'Main'
        elif row['HubID_collection'] != row['HubID_delivery'] and row['LMR_collection'] != 'Regional':
            return 'Main'
        elif row['HubID_collection'] == row['HubID_delivery'] and row['LMR_delivery'] != 'Regional':
            return 'Local'
        elif row['HubID_collection'] == row['HubID_delivery'] and row['LMR_collection'] != 'Regional':
            return 'Local'
        else:
            return 'Other'
    elif row['courier'] == 'courier_b':
        return row['LMR_delivery']

plans['overall_LMR'] = plans.apply(route_classification, axis=1)
#print(plans)

# Get courier cost

plans = pd.merge(plans, courier_costs_df, how='left', left_on=['delivery_type', 'courier', 'overall_LMR'], right_on=['delivery_type', 'courier_id', 'lmr'] )
plans = plans.drop(columns=['courier_id', 'lmr'])
print(plans)

#print(order_inventory)

       order_key                         allocation_id cancellations  \
0        order_1  018cd7ee-fab0-77d9-b417-61fd0f96e865          None   
1        order_2  018cd951-c168-7149-8434-1c66c1a9e5a0          None   
2        order_3  018cdb42-2356-7261-8393-9f18149c39d4          None   
3        order_4  018cda6a-5617-76ff-965b-2de978427181          None   
4        order_5  018cd939-895f-7fea-a0e3-0c79f800f6ab          None   
...          ...                                   ...           ...   
9469  order_3808  018cd8f1-d26a-7953-95ec-576390d1808f          None   
9470  order_3808  018cd8f1-d26a-7953-95ec-576390d1808f          None   
9471  order_3808  018cd8f1-d26a-7953-95ec-576390d1808f          None   
9472  order_3809  018cd935-f75e-7960-9aca-b862f8db603d          None   
9473  order_3810  018cd882-025b-78f6-a13c-4fc4a6a50f4a          None   

                          created_at  dry_run external_order_reference  \
0     2024-01-05T04:43:32.443897453Z    False              B4

In [283]:
# For each allocation, all items allocated to a particular facility can be packed into a single parcel. Courier costs are assigned per parcel.

# Get the courier cost for each batch of itmes colleced per branch
plans_pivot_cost = pd.pivot_table(plans, values='cost', index=['order_key', 'allocation_id', 'branch_number', 'courier'], aggfunc='min', fill_value=0)
#print(plans_pivot_cost.loc())

# Reset the index to have a flat DataFrame
plans_pivot_cost_reset = plans_pivot_cost.reset_index()
#print("\nPlans Cost Pivot Reset")
#print(plans_pivot_cost_reset)

# Get the list of items (SKUs) assoicated with each branch
plans_pivot_sku = pd.pivot_table(plans, values='sku', index=['order_key', 'allocation_id', 'branch_number', 'courier'], aggfunc=lambda x: ', '.join(x), fill_value=0)

# Reset the index to have a flat DataFrame
plans_pivot_sku_reset = plans_pivot_sku.reset_index()
#print("\nPlans SKUs Pivot Reset")
#print(plans_pivot_sku_reset)



# Join the two pivot tables to get the costs and the SKUs for each order
allocation_by_branch = pd.merge(plans_pivot_cost_reset, plans_pivot_sku_reset, how='right', left_on=['order_key', 'allocation_id', 'branch_number', 'courier'], right_on=['order_key', 'allocation_id', 'branch_number', 'courier'])

#print(allocation_by_branch)
print(allocation_by_branch.loc[allocation_by_branch['order_key'] == 'order_3779'])



       order_key                         allocation_id branch_number  \
4953  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        030310   
4954  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        030460   
4955  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        030467   
4956  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        030625   
4957  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        030658   
4958  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        030663   
4959  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        030682   
4960  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        030786   
4961  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        030805   
4962  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        030993   
4963  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        038924   
4964  order_3779  018cda4b-ee95-7d95-9bf7-99e2813ad407        039944   

         courier cost                                          

In [284]:
# Save to CSV files (optional)
allocation_by_branch.to_csv('/content/drive/MyDrive/Colab/Output/allocation_plans_current.csv', index=False)